In [3]:
%pip install azureml-sdk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


**Importing the dependencies**

In [15]:
import os
import json
import requests

from collections.abc import Iterable

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

In [16]:
# laoding the configuration file - standard way - use .env file and load_dotenv from python-dotenv module
config_file_path = "config.json"

# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]  
workspace_name = data["workspace_name"]
region = data["region"]

In [17]:
print(resource_group)
print(workspace_name)
print(region)

mlflow-rg
mlflow-ws
Central India


**Create a Resource Group from Azure Portal**

In [20]:
# Create a workspace

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

print(f'Workspace {workspace_name} created')

WorkspaceException: WorkspaceException:
	Message: Workspace with name 'mlflow-ws' already exists under resource group with name 'mlflow-rg'.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Workspace with name 'mlflow-ws' already exists under resource group with name 'mlflow-rg'."
    }
}

In [21]:
# Specify the path to your  model file
model_path = 'diabetes_model.pkl'

In [22]:
model_name='diabetes_prediction_model'

In [23]:
# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

Registering model diabetes_prediction_model


In [24]:
# Create a Conda environment for your scikit-learn model
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [25]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

In [26]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [27]:
service = Model.deploy(workspace=ws,
                       name='diabetes-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

C:\Users\Chinmay\AppData\Local\Temp\ipykernel_16928\3861565384.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-04-28 20:37:58+05:30 Creating Container Registry if not exists..
2024-04-28 20:47:58+05:30 Registering the environment..
2024-04-28 20:48:02+05:30 Building image..
2024-04-28 20:55:46+05:30 Generating deployment configuration.
2024-04-28 20:55:47+05:30 Submitting deployment to compute..
2024-04-28 20:55:54+05:30 Checking the status of deployment diabetes-prediction-service..
2024-04-28 20:57:08+05:30 Checking the status of inference endpoint diabetes-prediction-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [28]:
scoring_uri = service.scoring_uri

In [29]:
scoring_uri

'http://74e06c1d-21ff-4baa-901f-c9d31c9a201a.centralindia.azurecontainer.io/score'

**Cleaning up all the created resources**

In [30]:
# Delete the service

service_name = "diabetes-prediction-service"

service = ws.webservices[service_name]
service.delete()

Running
2024-04-28 20:59:54+05:30 Check and wait for operation (ed46da56-50ae-41cb-b736-e7942e2929ce) to finish.
2024-04-28 20:59:58+05:30 Deleting service entity.
Succeeded


In [31]:
# Specify the name of the registered model to delete
model_name = 'diabetes_prediction_model'

# Delete the model
registered_model = ws.models[model_name]
registered_model.delete()


In [32]:
# Delete the workspace
ws.delete(delete_dependent_resources=True)